# Web scrapping

In [1]:
# install.packages("dplyr")
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
# install.packages("rvest")
library(rvest)

## Code final : récupération des données parmis toutes les données de data.gouv

In [3]:
url <- "https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/"
download.file(url = url, "datagouv.html")
html <- read_html("datagouv.html")
html_c <- html_children(html)
html_wanted <- html %>% html_nodes('.card-body')
pos <- Position(x = html_wanted, f = function(x){ grepl("donnees-hospitalieres-covid19",html_text(x))})
html_wanted2 <- html_wanted[pos] %>% html_nodes('a') %>% html_attr('href')
l_html = length(html_wanted2)
link <- html_wanted2[l_html-1]

df_hebdo_covid <- read.csv(link, sep=";")
head(df_hebdo_covid,10)

dep,sexe,jour,hosp,rea,HospConv,SSR_USLD,autres,rad,dc
01,0,2020-03-18,2,0,NA,NA,NA,1,0
01,1,2020-03-18,1,0,NA,NA,NA,1,0
01,2,2020-03-18,1,0,NA,NA,NA,0,0
02,0,2020-03-18,41,10,NA,NA,NA,18,11
02,1,2020-03-18,19,4,NA,NA,NA,11,6
02,2,2020-03-18,22,6,NA,NA,NA,7,5
03,0,2020-03-18,4,0,NA,NA,NA,1,0
03,1,2020-03-18,1,0,NA,NA,NA,0,0
03,2,2020-03-18,3,0,NA,NA,NA,1,0
04,0,2020-03-18,3,1,NA,NA,NA,2,0


## Découpage des étapes

### Récupération du code html

In [4]:
# L'url de la bibliothèque des données
url <- "https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/"
download.file(url = url, "datagouv.html")
# Lecture du code html
html <- read_html("datagouv.html")
html

{html_document}
<html lang="fr">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body class="dataset-display theme-gouvfr">\r\n    <section id="app"><sec ...

### Découpage du code html

In [5]:
# Séparation du header et du body
html_c <- html_children(html)
html_c

{xml_nodeset (2)}
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body class="dataset-display theme-gouvfr">\r\n    <section id="app"><sec ...

In [6]:
# Extraction des cellules contenant les liens d'accès aux datasets
html_wanted <- html %>% html_nodes('.card-body')
html_wanted

{xml_nodeset (50)}
 [1] <div class="card-body">\r\n        <header class="card-header" id="resou ...
 [2] <div class="card-body">\r\n        <header class="card-header" id="resou ...
 [3] <div class="card-body">\r\n        <header class="card-header" id="resou ...
 [4] <div class="card-body">\r\n        <header class="card-header" id="resou ...
 [5] <div class="card-body">\r\n        <header class="card-header" id="resou ...
 [6] <div class="card-body">\r\n        <header class="card-header" id="resou ...
 [7] <div class="card-body">\r\n        <header class="card-header" id="resou ...
 [8] <div class="card-body">\r\n        <header class="card-header" id="resou ...
 [9] <div class="card-body">\r\n        <header class="card-header" id="resou ...
[10] <div class="card-body">\r\n        <header class="card-header" id="resou ...
[11] <div class="card-body">\r\n        <header class="card-header" id="resou ...
[12] <div class="card-body">\r\n        <header class="card-header" id="resou .

### Recherche d'éléments spécifiques

In [7]:
# Définition de la fonction de recherche d'un caractère dans une chaine de caractère
l <- grepl("donnees-hospitalieres-covid19",html_wanted %>% html_text())
head(l,10)

[1] FALSE FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE  TRUE  TRUE

In [8]:
# Recherche de la position de la premiere node contenant la chaine de caractère clef
pos <- Position(x = html_wanted, f = function(x){ grepl("donnees-hospitalieres-covid19",html_text(x))})
pos

[1] 6

### Récupération du lien d'accès au dataset

In [9]:
html_wanted2 <- html_wanted[pos] %>% html_nodes('a') %>% html_attr('href')
l_html = length(html_wanted2)
# Le dernier lien est toujours le plus stable du site internet
link <- html_wanted2[l_html-1]
link

[1] "https://static.data.gouv.fr/resources/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/20220113-190617/donnees-hospitalieres-covid19-2022-01-13-19h06.csv"

### Lecture et récupération du dataset

In [10]:
df_hebdo_covid <- read.csv(link, sep=";")
#### Changement du nom des colonnes
# names(data) <- c("X", "année", "prénom", "genre", "naissance")
# Affichage des 5 premières lignes
head(df_hebdo_covid,15)

dep,sexe,jour,hosp,rea,HospConv,SSR_USLD,autres,rad,dc
01,0,2020-03-18,2,0,NA,NA,NA,1,0
01,1,2020-03-18,1,0,NA,NA,NA,1,0
01,2,2020-03-18,1,0,NA,NA,NA,0,0
02,0,2020-03-18,41,10,NA,NA,NA,18,11
02,1,2020-03-18,19,4,NA,NA,NA,11,6
02,2,2020-03-18,22,6,NA,NA,NA,7,5
03,0,2020-03-18,4,0,NA,NA,NA,1,0
03,1,2020-03-18,1,0,NA,NA,NA,0,0
03,2,2020-03-18,3,0,NA,NA,NA,1,0
04,0,2020-03-18,3,1,NA,NA,NA,2,0
